Итак, мы примерно разобрались, как можно искать элементы на странице и фильтровать их по содержимому и значению аттрибутов, если нам удалось скачать HTML код

Но к сожалению, далеко не всегда сайты рады краулерам, и пытаются распознать, что к ним обращается машина, а не человек. Например, попробуем скачать главную страницу dotabuff.com.

In [1]:
import requests
 
answer = requests.get("https://ru.dotabuff.com/heroes")
print(answer.text)

<!DOCTYPE html>
<html>
<head>
  <title>DOTABUFF - Too Many Requests</title>
  <meta name="viewport" content="width=device-width, initial-scale=1"></meta>
  <style>
    html {
      background: #1C242D;
      color: #C4C4C4;
      font-family: "Lucida Grande", Tahoma, Verdana, Arial, sans-serif;
      font-size: 14px;
      width: 100%;
      height: 100%;
    }
    body {
      margin: 21% auto;
      text-align: center;
    }
    p a {
      color: #A9CF54;
      text-decoration: none
    }
    #logo {
      background: #ED3B1C;
      color: #FFFFFF;
      display: inline-block;
      font-family: Arial;
      font-weight: 800;
      padding: 6px 12px;
    }
    #logo a {
      color: #FFFFFF;
      text-decoration: none;
    }
    #status {
      font-weight: 200;
      color: #FFFFFF;
    }
  </style>
</head>
<body>
  <h1 id="logo"><a href="/">DOTABUFF</a></h1>
  <h2 id="status">Too Many Requests</h2>
  <p>You have been rate limited for making too many requests. Please try again lat

Вместо кода главной страницы мы видим подозрительное сообщение об ошибке. Попробуем подсмотреть, чем запрос из питона отличается от запроса браузера. Для этого можно заглянуть в отладчик в хроме (right-click - просмотр кода страницы - network, обновите страницу, выберите самый верхний запрос).

![](https://storage.geekclass.ru/images/e1e16399-0f18-4ffb-875e-5fab51e9fb1f.png)

Основная разница - в наличии заголовков (headers). Чтобы преодолеть защиту, можно попробовать добавить их к запросу по аналогии с браузером.

Начнем с самого простого - `user-agent` (это версия используемого браузера).

In [5]:
import requests

headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36"
}
 
answer = requests.get("https://ru.dotabuff.com/heroes", headers=headers)
# морока со скриптом - из за тонкостей отображения в гикклассе
print(answer.text.replace('script', 'скрипт')[:2000])


<!DOCTYPE html><html><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type" /><title>Герои - Все герои - DOTABUFF - Dota 2 Stats</title><link rel="stylesheet" media="all" href="/assets/application-4ee1beb1a0dc7e4bbf0b41c7392a346c411400c244f7ce244c95c504cc3364bf.css" /><link rel="alternate" type="application/rss+xml" title="RSS" href="https://ru.dotabuff.com/blog.rss" /><скрипт src="/assets/application-1101a895b15863c0d846939c638981db3aef22b34f0ea966e9aaea95b5acc313.js"></скрипт><скрипт src="https://k9v7.dotabuff.com/a.js" type="text/javaскрипт" async="async"></скрипт><meta name="csrf-param" content="authenticity_token" />
<meta name="csrf-token" content="dcKrOd008+M8/Prh30knbo9Q6hsf4laNaqRPWpst7yllMLI6BYGiS6zjuVEHGardwgnKEP5LEUHilN86JsUNJg==" /><meta name="keywords" content="дота 2 статистика, дота 2 статы, дота 2, статистика, дота" /><meta name="locale" content="ru" /><meta name="viewport" content="width=device-width, initial-scale=1" /><meta name="format-detection" 

Кажется, сработало! Можно даже вывести имена героев.

In [7]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(answer.text)

for hero in soup.find_all('div', {'class': 'hero'}):
    print(hero.text)

Abaddon 
Alchemist 
Ancient Apparition 
Anti-Mage 
Arc Warden 
Axe 
Bane 
Batrider 
Beastmaster 
Bloodseeker 
Bounty Hunter 
Brewmaster 
Bristleback 
Broodmother 
Centaur Warrunner 
Chaos Knight 
Chen 
Clinkz 
Clockwerk 
Crystal Maiden 
Dark Seer 
Dark Willow 
Dazzle 
Death Prophet 
Disruptor 
Doom 
Dragon Knight 
Drow Ranger 
Earth Spirit 
Earthshaker 
Elder Titan 
Ember Spirit 
Enchantress 
Enigma 
Faceless Void 
Grimstroke 
Gyrocopter 
Huskar 
Invoker 
Io 
Jakiro 
Juggernaut 
Keeper of the Light 
Kunkka 
Legion Commander 
Leshrac 
Lich 
Lifestealer 
Lina 
Lion 
Lone Druid 
Luna 
Lycan 
Magnus 
Mars 
Medusa 
Meepo 
Mirana 
Monkey King 
Morphling 
Naga Siren 
Nature's Prophet 
Necrophos 
Night Stalker 
Nyx Assassin 
Ogre Magi 
Omniknight 
Oracle 
Outworld Devourer 
Pangolier 
Phantom Assassin 
Phantom Lancer 
Phoenix 
Puck 
Pudge 
Pugna 
Queen of Pain 
Razor 
Riki 
Rubick 
Sand King 
Shadow Demon 
Shadow Fiend 
Shadow Shaman 
Silencer 
Skywrath Mage 
Slardar 
Slark 
Sniper 
Spectre 
S

Кстати, точно такой же трюк можно провернуть с авторизацией на сайте - достаточно добавить заголовок cookie и запроса в браузере.

Теперь попробуем скачать страницу с информацией о кроссовках - https://www.goat.com/sneakers.

In [10]:
import requests
 
answer = requests.get("https://www.goat.com/sneakers")
print(answer.text[:500])

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Access to this page has been denied.</title>
    <link href="https://fonts.googleapis.com/css?family=Open+Sans:300" rel="stylesheet">
    <style>
        html, body {
            margin: 0;
            padding: 0;
            font-family: 'Open Sans', sans-serif;
            color: #000;
        }
        a {
            color: #c5c5c5;
          


Судя по заголовку - доступ запрещен. Попробуем добавить заголовки.

In [17]:
import requests

headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36"
}
 
answer = requests.get("https://www.goat.com/sneakers", headers=headers)
# морока со скриптом - из за тонкостей отображения в гикклассе
print(answer.text.replace('script', 'скрипт')[:1000])

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8" />
    <link rel="icon" type="image/x-icon" href="/public/web-assets/icons/favicon.ico" />
    <meta name="p:domain_verify" content="70d0748bed9a38a632e92fac88c70291" />
    <meta name="google-site-verification" content="UKJdW4ASatmO3fEQRgy9oOKOWmy4Mi5DZqtu8vReGTs" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <link rel="stylesheet" type="text/css" href="https://static.goat.com/public/web-assets/fonts/GT-Sectra-Book.css" />
<link rel="stylesheet" type="text/css" href="https://static.goat.com/public/web-assets/fonts/GT-Sectra-Medium.css" />
<link rel="stylesheet" type="text/css" href="https://static.goat.com/public/web-assets/fonts/GT-Sectra-Regular.css" />
<link rel="stylesheet" type="text/css" href="https://static.goat.com/public/web-assets/fonts/PlainBold-Regular.css" />
<link rel="stylesheet" type="text/css" href="https://static.goat.com/public/web-assets/fonts/PlainLight-Regula

Страница скачалось, но попробуем найти в ней ссылки на кроссовки...

In [21]:
soup = BeautifulSoup(answer.text)

len(soup.find_all('a'))

0

Ссылок нет в принципе! Дело в том, что страница генерируется с помощью JavaScripta.

Что делать? Можно пробовать реверсить API, а можно просто доверить это браузеру. Буквально - завести себе собственный Google Chrome и управлять им через питон.

Для этого испольузется библиотека Selenium. С ее помощью можно не только выполнять JavaScript на странице, но и писать полноценные тесты для интерфейса - нажимать кнопки на странице и заполнять текстовые поля.

In [22]:
!pip install selenium

    100% |████████████████████████████████| 911kB 7.5MB/s ta 0:00:011


Отдельно потребуется скачать "драйвер" для хрома и положить его в любую папку из PATH. Например, в папку с анакондой.

https://sites.google.com/a/chromium.org/chromedriver/downloads

In [31]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get('https://www.goat.com/sneakers/')

soup = BeautifulSoup(driver.page_source)
len(soup.find_all('a'))


110

Сработало! Попробуем вывести имена кроссовок на первой странице...

![](https://storage.geekclass.ru/images/db85742e-5ef5-4fca-bfb5-0ebc12499762.png)

In [35]:
for cell in soup.find_all('a', {'class': 'cell'}):
    name = cell.find('div', {'class': 'grid-cell--name'})
    print(name.text)

AIR JORDAN 1 RETRO HIGH OG 'FEARLESS'
YEEZY BOOST 350 V2 'CREAM WHITE / TRIPLE WHITE'
AIR JORDAN 1 RETRO HIGH OG 'SHATTERED BACKBOARD 3.0'
BLUE THE GREAT X AIR JORDAN 1 MID 'FEARLESS'
YEEZY BOOST 350 V2 'ZEBRA'
AIR JORDAN 13 RETRO 'ISLAND GREEN'
YEEZY BOOST 350 V2 'BLACK NON-REFLECTIVE'
TRAVIS SCOTT X AIR JORDAN 1 RETRO HIGH OG 'MOCHA'
TRAVIS SCOTT X AIR FORCE 1 LOW 'CACTUS JACK'
YEEZY BOOST 700 'WAVE RUNNER'
AIR FORCE 1 '07 'WHITE'
AIR JORDAN 1 RETRO HIGH OG 'OBSIDIAN'
YEEZY 500 'SOFT VISION'
TRAVIS SCOTT X AIR JORDAN 6 RETRO 'OLIVE'
YEEZY BOOST 350 V2 'CLAY'
YEEZY BOOST 350 V2 'CLOUD WHITE NON-REFLECTIVE'
AIR JORDAN 4 RETRO OG 'BRED' 2019
OFF-WHITE X AIR JORDAN 1 RETRO HIGH OG 'CHICAGO'
KENDRICK LAMAR X REACT ELEMENT 55 'PURE PLATINUM'
AIR JORDAN 11 RETRO 'CONCORD' 2018
YEEZY BOOST 350 V2 'BLUE TINT'
SPONGEBOB SQUAREPANTS X KYRIE 5 'PINEAPPLE HOUSE'
OFF-WHITE X AIR JORDAN 1 RETRO HIGH OG 'UNC'
AIR FEAR OF GOD 1 'OATMEAL'
OLIVIA KIM X WMNS AIR JORDAN 4 RETRO 'NO COVER'
YEEZY BOOST 350

Теперь попробуем получить больше кроссовок просто кликнув по ссылке!

In [39]:
element = driver.find_element_by_xpath("//button[@data-qa='search_grid_see_more']");
element.click();

In [41]:
soup = BeautifulSoup(driver.page_source)
for cell in soup.find_all('a', {'class': 'cell'}):
    name = cell.find('div', {'class': 'grid-cell--name'})
    print(name.text)

AIR JORDAN 1 RETRO HIGH OG 'FEARLESS'
YEEZY BOOST 350 V2 'CREAM WHITE / TRIPLE WHITE'
AIR JORDAN 1 RETRO HIGH OG 'SHATTERED BACKBOARD 3.0'
BLUE THE GREAT X AIR JORDAN 1 MID 'FEARLESS'
YEEZY BOOST 350 V2 'ZEBRA'
AIR JORDAN 13 RETRO 'ISLAND GREEN'
YEEZY BOOST 350 V2 'BLACK NON-REFLECTIVE'
TRAVIS SCOTT X AIR JORDAN 1 RETRO HIGH OG 'MOCHA'
TRAVIS SCOTT X AIR FORCE 1 LOW 'CACTUS JACK'
YEEZY BOOST 700 'WAVE RUNNER'
AIR FORCE 1 '07 'WHITE'
AIR JORDAN 1 RETRO HIGH OG 'OBSIDIAN'
YEEZY 500 'SOFT VISION'
TRAVIS SCOTT X AIR JORDAN 6 RETRO 'OLIVE'
YEEZY BOOST 350 V2 'CLAY'
YEEZY BOOST 350 V2 'CLOUD WHITE NON-REFLECTIVE'
AIR JORDAN 4 RETRO OG 'BRED' 2019
OFF-WHITE X AIR JORDAN 1 RETRO HIGH OG 'CHICAGO'
KENDRICK LAMAR X REACT ELEMENT 55 'PURE PLATINUM'
AIR JORDAN 11 RETRO 'CONCORD' 2018
YEEZY BOOST 350 V2 'BLUE TINT'
SPONGEBOB SQUAREPANTS X KYRIE 5 'PINEAPPLE HOUSE'
OFF-WHITE X AIR JORDAN 1 RETRO HIGH OG 'UNC'
AIR FEAR OF GOD 1 'OATMEAL'
OLIVIA KIM X WMNS AIR JORDAN 4 RETRO 'NO COVER'
YEEZY BOOST 350

Ага! Теперь кроссовок стало в два раза больше. Можно зациклить операцию, пока количество не перестанет изменяться...